In [1]:
import pandas as pd
import numpy as np

In [2]:
#Review Hop Teaming Dataset
hop_review = pd.read_csv('data/DocGraph_Hop_Teaming_2017.csv', nrows = 50000)

In [3]:
hop_review.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1003863580,1000000004,19,19,108.895,84.598
1,1043250400,1000000004,20,20,87.000,77.173
2,1033239413,1000000004,20,20,58.800,76.982
3,1033142146,1000000004,491,535,10.232,36.558
4,1013957562,1000000004,25,26,78.692,59.305


Filter the hop_review dataset: transaction_count >= 50, average_day_wait < 50 


In [4]:
pd.to_numeric(hop_review.average_day_wait) 


0        108.895
1         87.000
2         58.800
3         10.232
4         78.692
          ...   
49995     79.917
49996     56.739
49997     55.286
49998     42.538
49999      8.750
Name: average_day_wait, Length: 50000, dtype: float64

In [5]:
hop_filter = hop_review[hop_review['transaction_count'] >= 50]
#hop_filter

In [6]:
hop_filter2 = hop_filter[hop_filter['average_day_wait'] < 50]
hop_filter2

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
3,1033142146,1000000004,491,535,10.232,36.558
6,1013977990,1003000126,134,145,27.352,51.137
8,1013996669,1003000126,91,92,35.152,68.009
10,1033102504,1003000126,52,64,15.328,38.300
15,1003029620,1003000126,111,121,33.058,58.981
...,...,...,...,...,...,...
49954,1033157821,1003834409,89,134,48.007,61.857
49967,1013268218,1003834490,127,142,7.000,28.671
49978,1033192554,1003834508,27,60,21.433,73.239
49983,1003913252,1003834508,39,61,24.475,69.104


In [7]:
hop_filter2 = hop_filter2.rename(columns={'from_npi':'npi'})

In [8]:
hop_filter2.head(3)

,npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
3,1033142146,1000000004,491,535,10.232,36.558
6,1013977990,1003000126,134,145,27.352,51.137
8,1013996669,1003000126,91,92,35.152,68.009


Resources: 
https://stackoverflow.com/questions/36684013/extract-column-value-based-on-another-column-pandas-dataframe
https://beginnersbook.com/2018/01/python-while-loop/


In [9]:
hop_filter2.dtypes

npi                    int64
to_npi                 int64
patient_count          int64
transaction_count      int64
average_day_wait     float64
std_day_wait         float64
dtype: object

Read in Kedir's filtered nppes dataset and merge with the hop_teaming dataset.  Run query using sql/sqlite3 

In [10]:
nppes = pd.read_csv('data/npp_taxo_cbsa_nash.csv')
nppes.head()

,npi,entity_type_code,replacement_npi,employer_identification_number_ein,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,...,notes,display_name,postal_code,zip,cbsa,res_ratio,bus_ratio,oth_ratio,tot_ratio,zip1
0,1134122187,1.0,NaN,NaN,NaN,RUDNICKE,CHERYL,DENISE,MRS.,NaN,...,NaN,NaN,37203,37203.0,34980.0,1.00000,1.000000,1.0,1.000000,37203
1,1003819046,1.0,NaN,NaN,NaN,NYLANDER,BARBARA,H,NaN,NaN,...,NaN,NaN,37203,37203.0,34980.0,1.00000,1.000000,1.0,1.000000,37203
2,1750384780,1.0,NaN,NaN,NaN,PERRIGIN,JULIE,A,DR.,NaN,...,NaN,NaN,37055,37055.0,34980.0,0.98654,0.997989,1.0,0.988105,37055
3,1922001957,1.0,NaN,NaN,NaN,PRESLEY,RICHARD,E,NaN,NaN,...,NaN,NaN,37203,37203.0,34980.0,1.00000,1.000000,1.0,1.000000,37203
4,1073516001,1.0,NaN,NaN,NaN,ROSS,DAVID,L,DR.,NaN,...,NaN,NaN,37055,37055.0,34980.0,0.98654,0.997989,1.0,0.988105,37055


In [11]:
nppes.dtypes

npi                                                          int64
entity_type_code                                           float64
replacement_npi                                            float64
employer_identification_number_ein                          object
provider_organization_name_legal_business_name              object
provider_last_name_legal_name                               object
provider_first_name                                         object
provider_middle_name                                        object
provider_name_prefix_text                                   object
provider_name_suffix_text                                   object
provider_credential_text                                    object
provider_first_line_business_practice_location_address      object
provider_second_line_business_practice_location_address     object
provider_business_practice_location_address_city_name       object
provider_business_practice_location_address_state_name      ob

In [12]:
import sqlite3

First, we need to connect to our database. The connect function will either create a new database if one does not already exist or connect to an existing one.

In [13]:
#create an empty df
db = sqlite3.connect('data/hop_nppes.sqlite')

Now, we can chunk through the data and for each row, add the rows to a table in our sqlite database. 
To keep track of how much progress has been made, we can use the `tqdm` library.

In [14]:
from tqdm.notebook import tqdm  
#tqdm is a function that helps with chunking by providing time bar 

In [15]:
db = sqlite3.connect('data/hop_nppes.sqlite')

for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2017.csv', chunksize = 10000)):
    chunk.to_sql('team', db, if_exists = 'append', index = False)
    
db.execute("CREATE INDEX from_npi ON team(from_npi)")    
    
db.close()
# 'team' is the new table name within large db called hop_teaming.sqlite

In [17]:
db = sqlite3.connect('data/hop_nppes.sqlite')

query_team = "SELECT * FROM team WHERE (transaction_count >= 50 AND average_day_wait < 50)"

team_filter_sqlite = pd.read_sql(query_team, db)

db.close()

In [18]:
team_filter_sqlite.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1033142146,1000000004,491,535,10.232,36.558
1,1013977990,1003000126,134,145,27.352,51.137
2,1013996669,1003000126,91,92,35.152,68.009
3,1033102504,1003000126,52,64,15.328,38.300
4,1003029620,1003000126,111,121,33.058,58.981


In [20]:
team_filter_sqlite.shape

(31704890, 6)

Bring in the nppes dataset into the hop_nppes as a table

In [23]:
db = sqlite3.connect('data/hop_nppes.sqlite')
for chunk in tqdm(pd.read_csv('data/npp_taxo_cbsa_nash.csv', chunksize = 10000)):
    chunk.to_sql('nppes', db, if_exists = 'append', index = False)
    
db.close()

Match team.from_npi to nppes.npi.  First create index on these two columns

In [24]:
db = sqlite3.connect('data/hop_nppes.sqlite')

db.execute('CREATE INDEX npi ON nppes(npi)')

db.close()



Merge the nppes dataset and the team dataset on 'npi'

In [28]:
query_merge = """
SELECT *  
FROM nppes AS n 
JOIN team AS t 
ON n.npi = t.from_npi;
 """
 


In [29]:
with sqlite3.connect('data/hop_nppes.sqlite') as db: 
    team_nppes = pd.read_sql(query_merge, db)



In [30]:
team_nppes.shape

(1571419, 42)

In [31]:
team_nppes.head()

,npi,entity_type_code,replacement_npi,employer_identification_number_ein,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,...,bus_ratio,oth_ratio,tot_ratio,zip1,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1003863580,2.0,None,<UNAVAIL>,"ASSOCIATED PATHOLOGISTS, LLC",None,None,None,None,None,...,1.0,1.0,1.0,37027,1003863580,1000000004,19,19,108.895,84.598
1,1023223898,1.0,None,None,None,WOODFORD,RANDALL,L,None,None,...,1.0,1.0,1.0,37027,1023223898,1003000126,36,37,56.946,76.122
2,1003863580,2.0,None,<UNAVAIL>,"ASSOCIATED PATHOLOGISTS, LLC",None,None,None,None,None,...,1.0,1.0,1.0,37027,1003863580,1003000126,96,101,52.208,76.874
3,1003863580,2.0,None,<UNAVAIL>,"ASSOCIATED PATHOLOGISTS, LLC",None,None,None,None,None,...,1.0,1.0,1.0,37027,1003863580,1003000308,12,12,51.750,45.167
4,1003863580,2.0,None,<UNAVAIL>,"ASSOCIATED PATHOLOGISTS, LLC",None,None,None,None,None,...,1.0,1.0,1.0,37027,1003863580,1003001066,109,134,9.612,35.369


In [32]:
team_nppes.columns

Index(['npi', 'entity_type_code', 'replacement_npi',
       'employer_identification_number_ein',
       'provider_organization_name_legal_business_name',
       'provider_last_name_legal_name', 'provider_first_name',
       'provider_middle_name', 'provider_name_prefix_text',
       'provider_name_suffix_text', 'provider_credential_text',
       'provider_first_line_business_practice_location_address',
       'provider_second_line_business_practice_location_address',
       'provider_business_practice_location_address_city_name',
       'provider_business_practice_location_address_state_name',
       'provider_business_practice_location_address_postal_code',
       'provider_gender_code', 'taxonomy', 'code', 'grouping',
       'classification', 'specialization', 'definition', 'effective_date',
       'deactivation_date', 'last_modified_date', 'notes', 'display_name',
       'postal_code', 'zip', 'cbsa', 'res_ratio', 'bus_ratio', 'oth_ratio',
       'tot_ratio', 'zip1', 'from_npi', 'to

Clean up file to begin merge to assign entity code to from_npi

In [44]:
team_nppes_from = team_nppes.to_csv('data/team_nppes.csv')

In [55]:
db.close()

In [54]:
team_nppes_from = pd.read_csv('data/team_nppes.csv', index_col = 0)
team_nppes_from.head(3)

C:\Users\daily\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (19,20,21,22,23,24,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\daily\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,npi,entity_type_code,replacement_npi,employer_identification_number_ein,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,...,bus_ratio,oth_ratio,tot_ratio,zip1,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1003863580,2.0,NaN,<UNAVAIL>,"ASSOCIATED PATHOLOGISTS, LLC",NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,37027,1003863580,1000000004,19,19,108.895,84.598
1,1023223898,1.0,NaN,NaN,NaN,WOODFORD,RANDALL,L,NaN,NaN,...,1.0,1.0,1.0,37027,1023223898,1003000126,36,37,56.946,76.122
2,1003863580,2.0,NaN,<UNAVAIL>,"ASSOCIATED PATHOLOGISTS, LLC",NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,37027,1003863580,1003000126,96,101,52.208,76.874
3,1003863580,2.0,NaN,<UNAVAIL>,"ASSOCIATED PATHOLOGISTS, LLC",NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,37027,1003863580,1003000308,12,12,51.750,45.167
4,1003863580,2.0,NaN,<UNAVAIL>,"ASSOCIATED PATHOLOGISTS, LLC",NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,37027,1003863580,1003001066,109,134,9.612,35.369


In [61]:
team_nppes_from = team_nppes_from[team_nppes_from['entity_type_code']== 1]
team_nppes_from.shape

(922957, 42)

In [68]:
team_nppes_from_col = team_nppes_from.drop(columns = ['replacement_npi','employer_identification_number_ein',
                                       'provider_business_practice_location_address_city_name',
                                       'provider_business_practice_location_address_state_name',
                                       'provider_business_practice_location_address_postal_code',
                                       'provider_gender_code', 'effective_date', 'deactivation_date', 
                                       'last_modified_date','notes', 'display_name', 'postal_code', 'zip', 
                                       'cbsa', 'res_ratio', 'bus_ratio', 'oth_ratio'], axis = 1)

In [71]:
team_nppes_from_col.head(3)

,npi,entity_type_code,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,...,specialization,definition,tot_ratio,zip1,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
1,1023223898,1.0,NaN,WOODFORD,RANDALL,L,NaN,NaN,MD,5301 VIRGINIA WAY,...,NaN,NaN,1.0,37027,1023223898,1003000126,36,37,56.946,76.122
7,1033109285,1.0,NaN,NISBET,JOHN,J,DR.,NaN,MD,3024 BUSINESS PARK CIR,...,NaN,NaN,1.0,37072,1033109285,1003007204,17,17,74.118,90.385
10,1033109285,1.0,NaN,NISBET,JOHN,J,DR.,NaN,MD,3024 BUSINESS PARK CIR,...,NaN,NaN,1.0,37072,1033109285,1003010687,36,36,89.444,70.271


In [72]:
db = sqlite3.connect('data/hop_nppes.sqlite')

db.execute("CREATE INDEX to_npi ON team(to_npi)")

db.close()



In [74]:
# now merge the nppes dataset with the to_npi of the nppes dataset
query_merge_to = """
SELECT *  
FROM nppes AS n 
JOIN team AS t 
ON n.npi = t.to_npi;
"""

In [75]:
with sqlite3.connect('data/hop_nppes.sqlite') as db: 
    team_nppes_to = pd.read_sql(query_merge_to, db)

In [76]:
db.close()

In [77]:
team_nppes_to = team_nppes_to[team_nppes_to['entity_type_code']== 2]

In [78]:
team_nppes_to.shape

(650980, 42)

Now make a final csv for both the npi_to dataset and the nip_from dataset and start a new notebook to work with them

In [79]:
team_nppes_to.to_csv('data/team_nppes_to.csv')

In [80]:
team_nppes_from_col.to_csv('data/team_nppes_from.csv')